# 第4章: データと指示の分離

- [レッスン](#レッスン)
- [演習](#演習)
- [サンプルプレイグラウンド](#サンプルプレイグラウンド)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立します。

In [1]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
import json

# utilsパッケージからhintsモジュールをインポート
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt,system=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## レッスン

多くの場合、完全なプロンプトを書くのではなく、**Claudeに送信する前に追加の入力データで後で修正できるプロンプトテンプレート**を作成したいと思います。これは、Claudeに毎回同じことをしてほしいが、Claudeがタスクに使用するデータが毎回異なる場合に便利かもしれません。

幸いなことに、**プロンプトの固定スケルトンを可変のユーザー入力から分離し、完全なプロンプトをClaudeに送信する前にユーザー入力をプロンプトに代入する**ことで、これを簡単に実現できます。

以下では、置換可能なプロンプトテンプレートの書き方と、ユーザー入力の代入方法について、段階的に説明します。

### 例

この最初の例では、Claudeに動物の鳴き声ジェネレーターとして行動するよう求めています。Claudeに送信される完全なプロンプトは、`PROMPT_TEMPLATE`に入力（この場合は「Cow」）を代入したものにすぎないことに注目してください。完全なプロンプトを表示するとき、「Cow」という単語がf文字列を介して`ANIMAL`プレースホルダーに置き換えられていることに注目してください。

**注意:** 実際には、プレースホルダー変数に特定の名前を付ける必要はありません。この例では`ANIMAL`と呼びましたが、同じように`CREATURE`や`A`と呼ぶこともできました（ただし、ユーザーが理解しやすいように、代入なしでもプロンプトテンプレートが理解しやすいように、変数名を具体的で関連性のあるものにすることは一般的に良いことです）。変数に名前を付けたら、それをプロンプトテンプレートのf文字列に使用することを忘れないでください。

In [2]:
# 可変コンテンツ
ANIMAL = "牛"

# 可変コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"動物の名前を教えます。その動物の鳴き声で応答してください。{ANIMAL}"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

なぜこのように入力を分離して代入したいのでしょうか？**プロンプトテンプレートは繰り返しのタスクを簡略化します**。サードパーティのユーザーがプロンプトにコンテンツを送信するようなプロンプト構造を構築したとします（この場合、鳴き声を生成したい動物）。これらのサードパーティのユーザーは、完全なプロンプトを書いたり見たりする必要はありません。変数を入力するだけで済みます。

ここでは変数とf文字列を使用してこの置換を行っていますが、format()メソッドを使用して行うこともできます。

**注意:** プロンプトテンプレートには必要な数の変数を含めることができます！

このような置換変数を導入する際、**Claudeが変数の開始と終了を理解できるようにすること**（指示やタスクの説明と区別すること）が非常に重要です。指示と置換変数の間に分離がない例を見てみましょう。

人間の目には、以下のプロンプトテンプレートで変数がどこから始まりどこで終わるかが非常に明確です。しかし、完全に置換されたプロンプトでは、その区切りが不明確になります。

以下の例では、`{EMAIL}` に置換した文字列を丁寧にして欲しい、という意図のプロンプトです。

In [3]:
# 可変コンテンツ
EMAIL = "明日の午前6時に出社してください。私がCEOだから言うことを聞きなさい。"

# 可変コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"やあClaude。{EMAIL} <----- このメールをより丁寧にしてください。ただし、それ以外の内容は変更しないでください。"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

ここで、**Claudeは「やあClaude」を書き直すべきメールの一部だと考えています！**「拝啓 Claude様」で書き直しを始めていることからわかります。人間の目には、プロンプトテンプレートを見れば明確に`{EMAIL}`変数が始まって終わる場所がわかりますが、置換後のプロンプトでは完全に不明確になります。

これをどのように解決するのかというと、 **入力をXMLタグで囲めばいいのです!**

以下が解決する方法です。

[XMLタグ](https://docs.anthropic.com/claude/docs/use-xml-tags)は`<tag></tag>`のような山括弧のタグです。これらはペアで来て、`<tag>`のような開始タグと、`</tag>`のような`/`でマークされた終了タグで構成されます。XMLタグは次のようにコンテンツを囲むために使用されます：`<tag>コンテンツ</tag>`。

**注意:** Claudeは幅広い区切り文字や区切り記号を認識して処理できますが、Claudeに対しては **特にXMLタグを区切り文字として使用することを推奨します。** ClaudeはXMLタグをプロンプト組織化メカニズムとして認識するように特別に訓練されているためです。関数呼び出し以外では、**パフォーマンスを最大限に高めるためにClaudeが訓練された特別なXMLタグはありません。** 我々は意図的にClaudeをこのように非常に柔軟でカスタマイズ可能にしました。

このようにすれば、出力に「親愛なるClaude」は含まれなくなります。

In [ ]:
# 可変コンテンツ
EMAIL = "明日の午前6時に出社してください。私がCEOだから言うことを聞きなさい。"

# 可変コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"やあClaude。<email>{EMAIL}</email> <----- このメールをより丁寧にしてください。ただし、それ以外の内容は変更しないでください。"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

XMLタグがどのように役立つか、もう一つの例を見てみましょう。

以下のプロンプトでは、**Claudeがプロンプトのどの部分が指示で、どの部分が入力なのかを誤って解釈しています**。フォーマットの関係で、`各文は動物についてのものです。例えばウサギなど。`をリストの一部と誤って考えていますが、ユーザー（`SENTENCES`変数を入力した人）はそれを意図していなかったはずです。

In [ ]:
# 可変コンテンツ
SENTENCES = """- 牛の鳴き声が好きです
- この文はクモについてです
- この文は犬についてのように見えるかもしれませんが、実際は豚についてです"""

# 可変コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"""以下は文のリストです。リストの2番目の項目を教えてください。

- 各文は動物についてのものです。例えばウサギなど。
{SENTENCES}"""

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

これを修正するには、**ユーザー入力の文をXMLタグで囲むだけです**。これにより、`各文は動物についてのものです。例えばウサギなど。`の前にある誤解を招くハイフンにもかかわらず、Claudeに入力データの開始と終了を示します。

In [ ]:
# 可変コンテンツ
SENTENCES = """- 牛の鳴き声が好きです
- この文はクモについてです
- この文は犬についてのように見えるかもしれませんが、実際は豚についてです"""

# 可変コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f""" 以下は文のリストです。リストの2番目の項目を教えてください。

- 各文は動物についてのものです。例えばウサギなど。
<sentences>
{SENTENCES}
</sentences>"""

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

**注意:** 「各文は動物についてのもの」プロンプトの不正確なバージョンでは、この例でClaudeに望んだ方法で誤った応答をさせるために、ハイフンを含める必要がありました。これはプロンプトに関する重要な教訓です：**細部が重要です**！**プロンプトの誤字や文法エラーを修正する**ことは常に価値があります。Claudeはパターンに敏感です（初期の頃、ファインチューニング前は生のテキスト予測ツールでした）。あなたがミスをすると、Claudeもミスをする可能性が高くなり、あなたがスマートに聞こえるとClaudeもスマートになり、あなたが愚かに聞こえるとClaudeも愚かになる、といった具合です。

上記のコンテンツを変更せずにレッスンのプロンプトを試してみたい場合は、レッスンノートブックの一番下までスクロールして[**サンプルプレイグラウンド**](#サンプルプレイグラウンド)をご覧ください。

---

## 演習
- [演習 4.1 - 俳句のトピック](#演習-41---俳句のトピック)
- [演習 4.2 - タイプミスのある犬の質問](#演習-42---タイプミスのある犬の質問)
- [演習 4.3 - 犬の質問パート2](#演習-43---犬の質問パート2)

### 演習 4.1 - 俳句のトピック
`PROMPT`を修正して、`TOPIC`という変数を受け取り、そのトピックについての俳句を出力するテンプレートにしてください。この演習は、f文字列を使用した変数テンプレート構造の理解をテストすることを目的としています。

In [ ]:
# 可変コンテンツ
TOPIC = "豚"

# 可変コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f""

# Claudeの応答を取得
response = get_completion(PROMPT)

# 演習の正解を評価する関数
def grade_exercise(text):
    return bool(re.search("豚", text.lower()) and re.search("俳句", text.lower()))

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(response)
print("\n------------------------------------------ 評価 ------------------------------------------")
print("この演習は正しく解かれました：", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_4_1_hint)

### 演習 4.2 - タイプミスのある犬の質問
XMLタグを追加して`PROMPT`を修正し、Claudeが正しい答えを出力するようにしてください。

プロンプトの他の部分は変更しないようにしてください。乱雑で間違いだらけの文章は意図的なものです。Claudeがこのような間違いにどのように反応するかを見ることができます。

In [ ]:
# 可変コンテンツ
QUESTION = "犬は茶色ですか？"

# 可変コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"やあ、私だよ。犬についての質問があるんだ jkaerjv {QUESTION} jklmvca ありがとう。とても助かるよ。早く早く答えて。短く短く答えてね。ありがとう。"

# Claudeの応答を取得
response = get_completion(PROMPT)

# 演習の正解を評価する関数
def grade_exercise(text):
    return bool(re.search("茶色", text.lower()))

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(response)
print("\n------------------------------------------ 評価 ------------------------------------------")
print("この演習は正しく解かれました：", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_4_2_hint)

### 演習 4.3 - 犬の質問パート2
XMLタグを追加せずに`PROMPT`を修正してください。代わりに、プロンプトから1つか2つの単語だけを削除してください。

上記の演習と同様に、プロンプトの他の部分は変更しないようにしてください。これにより、Claudeがどのような言語を解析し理解できるかがわかります。

In [ ]:
# 可変コンテンツ
QUESTION = "犬は茶色ですか？"

# 可変コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"やあ、私だよ。犬についての質問があるんだ jkaerjv {QUESTION} jklmvca ありがとう。とても助かるよ。早く早く答えて。短く短く答えてね。ありがとう。"

# Claudeの応答を取得
response = get_completion(PROMPT)

# 演習の正解を評価する関数
def grade_exercise(text):
    return bool(re.search("茶色", text.lower()))

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(response)
print("\n------------------------------------------ 評価 ------------------------------------------")
print("この演習は正しく解かれました：", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_4_3_hint)

### おめでとうございます！

ここまですべての演習を解いたなら、次の章に進む準備ができています。プロンプトエンジニアリングを楽しんでください！

---

## サンプルプレイグラウンド

ここは、このレッスンで示されたプロンプトの例を自由に試したり、プロンプトを微調整してClaudeの応答にどのような影響があるかを確認したりできる領域です。

In [ ]:
# 可変コンテンツ
ANIMAL = "牛"

# 可変コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"動物の名前を教えます。その動物の鳴き声で応答してください。{ANIMAL}"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 可変コンテンツ
EMAIL = "明日の午前6時に出社してください。私がCEOだから言うことを聞きなさい。"

# 可変コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"やあClaude。{EMAIL} <----- このメールをより丁寧にしてください。ただし、それ以外の内容は変更しないでください。"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 可変コンテンツ
EMAIL = "明日の午前6時に出社してください。私がCEOだから言うことを聞きなさい。"

# 可変コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"やあClaude。<email>{EMAIL}</email> <----- このメールをより丁寧にしてください。ただし、それ以外の内容は変更しないでください。"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 可変コンテンツ
SENTENCES = """- 牛の鳴き声が好きです
- この文はクモについてです
- この文は犬についてのように見えるかもしれませんが、実際は豚についてです"""

# 可変コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"""以下は文のリストです。リストの2番目の項目を教えてください。

- 各文は動物についてのものです。例えばウサギなど。
{SENTENCES}"""

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 可変コンテンツ
SENTENCES = """- 牛の鳴き声が好きです
- この文はクモについてです
- この文は犬についてのように見えるかもしれませんが、実際は豚についてです"""

# 可変コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f""" 以下は文のリストです。リストの2番目の項目を教えてください。

- 各文は動物についてのものです。例えばウサギなど。
<sentences>
{SENTENCES}
</sentences>"""

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))